In [2]:
from metakernel import register_ipython_magics
register_ipython_magics()

import ipywidgets as widgets
import sys
from IPython.display import display
from IPython.display import clear_output

from pythonds.basic import Queue


def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "Richtig." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "Fail. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="submit")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])


frage1 ='Was ist das Ein/Ausgabeprinzip beim ADT Queue?'
frage2 ="""Was ist der Inhalt der Queue bei folgenden Operationen?
q = Queue()
q.enqueue(hello)
m.enqueue(dog)
m.enqueue(3)
m.dequeue()
"""

Q1 = create_multipleChoice_widget(frage1,['First-In, First-Out (FIFO)','Last-In, First-Out (LIFO)','Last-Out, First-In (LOFI)'],'First-In, First-Out (FIFO)')
Q2 = create_multipleChoice_widget(frage2, ['hello,dog','dog,3','hello,3','hello,dog,3'],'dog,3')

[<img src="Bilder/MIREVIBanner.jpg">](../../FMA_start_here.ipynb)

<!--[Hauptmenü](/notebooks/FMA_start_here.ipynb)-->

# Abstrakte Datenstrukturen, Teil 4

Teil 4 der interaktiven FMA-Lerneinheit zu abstrakten Datenstrukturen

## Ziele

- die abstrakten Datentypen Stack, Queue, Deque und List verstehen.
- die ADTs Stack, Queue und Deque unter Verwendung von Python-Listen implementieren können.
- die Performanz der Implementierungen von grundlegenden linearen Datenstrukturen zu verstehen.
- Verstehen der Präfix-, Infix- und Postfix-Ausdrucksformate.
- Verwendung von Stacks zur Auswertung von Postfix-Ausdrücken.
- Verwendung von Stacks zur Konvertierung von Ausdrücken von Infix- in Postfix-Ausdrücke.
- Verwendung von Warteschlangen für grundlegende Timing-Simulationen.
- Probleme erkennen, bei denen Stacks, Warteschlangen und Deques geeignete Datenstrukturen sind.
- **den ADT Liste als verlinkte Liste unter Verwendung des Knoten- und Zeigerkonzepts implementieren zu können.**
- **die Leistung der Implementierung der verknüpften Liste mit der Listenimplementierung von Python vergleichen.**
---


## Listen
Während der gesamten Diskussion der grundlegenden Datenstrukturen haben wir *Python-Listen* verwendet, um die vorgestellten abstrakten Datentypen zu implementieren. Die Liste ist ein leistungsfähiger und dennoch einfacher Sammelmechanismus, der dem Programmierer eine Vielzahl von Operationen zur Verfügung stellt. Allerdings enthalten nicht alle Programmiersprachen eine Listensammlung. In diesen Fällen muss der Begriff der Liste vom Programmierer implementiert werden.

Eine **Liste** ist eine Sammlung von Elementen, bei der jedes Element eine relative Position in Bezug auf die anderen Elemente einnimmt. Genauer gesagt werden wir diese Art von Liste als ungeordnete Liste bezeichnen. Wir können die Liste als eine Liste mit einem ersten Punkt, einem zweiten Punkt, einem dritten Punkt und so weiter betrachten. Wir können uns auch auf den Anfang der Liste (den ersten Punkt) oder das Ende der Liste (den letzten Punkt) beziehen. Der Einfachheit halber gehen wir davon aus, dass Listen keine doppelten Einträge enthalten können.

Zum Beispiel könnte die Sammlung der ganzen Zahlen 54, 26, 93, 17, 77 und 31 eine einfache ungeordnete Liste von Prüfungsergebnissen darstellen. Beachten Sie, dass wir sie als kommagetrennte Werte geschrieben haben, eine übliche Art, die Listenstruktur darzustellen. Natürlich würde Python diese Liste als [54,26,93,17,77,31] anzeigen.

---
### Der abstrakte Datentyp der ungeordneten Liste

Die Struktur einer ungeordneten Liste, wie oben beschrieben, ist eine Sammlung von Elementen, bei der jedes Element eine relative Position in Bezug auf die anderen einnimmt. Einige mögliche ungeordnete Listenoperationen sind nachstehend aufgeführt:

* <code>List()</code> erstellt eine neue Liste, die leer ist. Sie benötigt keine Parameter und gibt eine leere Liste zurück.
* <code>add(item)</code> fügt der Liste ein neues Element hinzu, wenn es noch nicht in der Liste enthalten ist.
* <code>remove(item)</code> entfernt das Element aus der Liste. Es benötigt das Element und modifiziert die Liste. Voraussetzung ist, dass das Element ist in der Liste vorhanden ist.
* <code>search(item)</code> sucht nach dem Element in der Liste. Es benötigt das Element und gibt einen booleschen Wert zurück.
* <code>isEmpty()</code> testet, ob die Liste leer ist. Gibt einen booleschen Wert zurück.
* <code>size()</code> gibt die Anzahl der Elemente in der Liste zurück. Benötigt keine Parameter und gibt eine ganze Zahl zurück.
* <code>append(item)</code> fügt ein neues Element am Ende der Liste hinzu, wodurch es zum letzten Element in der Sammlung wird. Es benötigt das Element und gibt nichts zurück. Vorraussetzung ist,dass das Objekt nicht bereits in der Liste ist.
* <code>index(item)</code> gibt die Position des Elements in der Liste zurück. Es benötigt das Element und gibt den Index zurück. Vorraussetzung ist, dass sich das Element in der Liste befindet.
* <code>anser(pos, item)</code> fügt der Liste an Position pos ein neues Element hinzu. Es benötigt das Element und gibt nichts zurück. Angenommen, das Element ist nicht bereits in der Liste und es gibt genügend vorhandene Elemente für die Position pos.
* <code>pop()</code> entfernt und gibt das letzte Element in der Liste zurück. Es benötigt nichts und gibt ein Element zurück. Angenommen, die Liste enthält mindestens ein Element.
* <code>pop(pos)</code> entfernt und gibt das Element an der Position pos zurück. Es benötigt die Position und gibt das Element zurück. Angenommen, das Element befindet sich in der Liste.

---
### Implementierung einer ungeordneten Liste: Verknüpfte Listen

Um eine ungeordnete Liste zu implementieren, werden wir eine so genannte **verknüpfte Liste** erstellen. Es sei daran erinnert, dass wir sicher sein müssen, dass wir die relative Positionierung der Elemente beibehalten können. Es ist jedoch nicht erforderlich, dass wir uns diese Positionierung merken. Betrachten wir zum Beispiel die in Abbildung 1 gezeigte Sammlung von Gegenständen. Es scheint, dass diese Werte zufällig platziert wurden. Wenn wir einige explizite Informationen in jedem Element beibehalten können, nämlich die Position des nächsten Elements (siehe Abbildung 2), dann kann die relative Position jedes Elements ausgedrückt werden, indem man einfach dem Link von einem Element zum nächsten folgt.

<img src="Bilder/ADT/adt_lists_fig1.PNG" width=400px>

<center><b>Abbildung 1:</b> Items, die in ihrer physischen Platzierung nicht eingeschränkt sind.</center>
<img src="Bilder/ADT/adt_lists_fig2.PNG" width=400px>

<center><b>Abbildung 2:</b> Relative Positionen, die durch explizite Links gehalten werden.</center>

Es ist wichtig zu beachten, dass die Position des ersten Punktes der Liste explizit angegeben werden muss. Sobald wir wissen, wo sich der erste Punkt befindet, kann uns der erste Punkt sagen, wo sich der zweite befindet, und so weiter. Die externe Referenz wird oft als **Kopf** der Liste bezeichnet. Ebenso muss der letzte Punkt wissen, dass es keinen nächsten Punkt gibt.

---
### Die <code>Node</code> Klasse
Der Grundbaustein für die Implementierung der verknüpften Liste ist der **Knoten**. Jedes Knotenobjekt muss mindestens zwei Informationen enthalten. Zunächst muss der Knoten das Listenelement selbst enthalten. Wir werden dies das **Datenfeld** des Knotens nennen. Darüber hinaus muss jeder Knoten eine Referenz auf den nächsten Knoten enthalten. Listing 1 zeigt die Python-Implementierung. Um einen Knoten zu konstruieren, müssen Sie den initialen Datenwert für den Knoten liefern. Die Auswertung der nachstehenden Zuweisungsanweisung ergibt ein Knotenobjekt mit dem Wert 93 (siehe Abbildung 3). Beachten Sie, dass wir typischerweise ein Knotenobjekt darstellen werden, wie in Abbildung 4 gezeigt. Die <code>Node</code>-Klasse enthält auch die üblichen Methoden zum Zugriff auf und zur Änderung der Daten und der nächsten Referenz.

**Listing 1**

In [3]:
class Node:
    def __init__(self,initdata):
        self.data = initdata
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self,newdata):
        self.data = newdata

    def setNext(self,newnext):
        self.next = newnext

Wir erstellen <code>Node</code>-Objekte auf die übliche Art und Weise:

In [4]:
temp = Node(93)
temp.getData()

93

Der spezielle Python-Referenzwert <code>None</code> wird in der <code>Node</code>-Klasse und später in der verknüpften Liste selbst eine wichtige Rolle spielen. Ein Verweis auf <code>None</code> wird bezeichnen, dass es keinen nächsten Knoten gibt. Beachten Sie im Konstruktor, dass ein Knoten zunächst mit <code>next</code> auf <code>None</code> erstellt wird. Da dies manchmal als "Erdung des Knotens" bezeichnet wird, verwenden wir das Standard-Erdungs-Symbol, um eine Referenz zu kennzeichnen, die sich auf <code>None</code> bezieht. Es ist immer eine gute Idee, Ihren anfänglichen nächsten Referenzwerten explizit <code>None</code> zuzuweisen.

<img src="Bilder/ADT/adt_lists_fig3.PNG" width=400px>

<center><b>Abbildung 3:</b> Knotenobjekt enthält das Element und einen Verweis auf den nächsten Knoten</center>
<img src="Bilder/ADT/adt_lists_fig4.PNG" width=300px>

<center><b>Abbildung 4:</b> Eine typische Darstellung für einen Knoten</center>

---
## Die <code>Unordered List</code>-Klasse

Wie wir oben vorgeschlagen haben, wird die ungeordnete Liste aus einer Sammlung von Knoten aufgebaut, von denen jeder mit dem nächsten durch explizite Verweise verbunden ist. Solange wir wissen, wo der erste Knoten (der den ersten Eintrag enthält) zu finden ist, kann jeder Eintrag danach durch sukzessives Folgen der nächsten Links gefunden werden. In diesem Sinne muss die Klasse <code>UnorderedList</code> eine Referenz auf den ersten Knoten beibehalten. Listing 2 zeigt den Konstruktor. Beachten Sie, dass jedes Listenobjekt eine einzige Referenz auf den Kopf der Liste beibehält.

**Listing 2**

In [5]:
class UnorderedList:

    def __init__(self):
        self.head = None

    def add(self,item):
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def isEmpty(self):
        return self.head == None
    
    def size(self):
        current = self.head
        count = 0
        while current != None:
            count = count + 1
            current = current.getNext()

        return count

    def search(self,item):
        current = self.head
        found = False
        while current != None and not found:
            if current.getData() == item:
                found = True
            else:
                current = current.getNext()

        return found

Wenn wir eine Liste erstellen, gibt es zunächst keine Punkte. Die Zuweisungsanweisung

In [6]:
mylist = UnorderedList()

erstellt die in Abbildung 5 gezeigte verknüpfte Listendarstellung. Wie wir in der Klasse <code>Node</code> diskutiert haben, wird der spezielle Verweis <code>None</code> wieder verwendet, um anzugeben, dass sich der Kopf der Liste auf nichts bezieht. Schließlich wird die zuvor angegebene Beispielliste durch eine verknüpfte Liste dargestellt, wie in Abbildung 6 gezeigt. Der Kopf der Liste bezieht sich auf den ersten Knoten, der den ersten Punkt der Liste enthält. Dieser Knoten wiederum enthält einen Verweis auf den nächsten Knoten (den nächsten Punkt) und so weiter. Es ist sehr wichtig zu beachten, dass die Listenklasse selbst keine Knotenobjekte enthält. Stattdessen enthält sie einen einzigen Verweis nur auf den ersten Knoten in der verknüpften Struktur.

<img src="Bilder/ADT/adt_lists_fig5.PNG" width=400px>

<center><b>Abbildung 5:</b> Eine leere Liste</center>
<img src="Bilder/ADT/adt_lists_fig6.PNG" width=500px>

<center><b>Abbildung 6:</b> Eine verknüpfte Liste von ganzen Zahlen</center>

Die in Listing 3 dargestellte Methode <code>isEmpty</code> prüft lediglich, ob der Listenkopf einen Verweis auf <code>None</code> enthält. Das Ergebnis des booleschen Ausdrucks <code>self.head==None</code> ist nur dann wahr, wenn es keine Knoten in der verknüpften Liste gibt. Da eine neue Liste leer ist, müssen der Konstruktor und die Prüfung auf leer miteinander konsistent sein. Dies zeigt den Vorteil der Verwendung des Verweises <code>None</code>, um das "Ende" der verknüpften Struktur zu bezeichnen. In Python kann <code>None</code> mit jeder Referenz verglichen werden. Zwei Referenzen sind gleich, wenn sie sich beide auf dasselbe Objekt beziehen. Wir werden dies in unseren übrigen Methoden häufig verwenden.

**Listing 3**

In [7]:
def isEmpty(self):
    return self.head == None

Wie bekommen wir also Einträge auf unsere Liste? Wir müssen die <code>add</code>-Methode implementieren. Bevor wir das jedoch tun können, müssen wir uns mit der wichtigen Frage befassen, wo in der verknüpften Liste das neue Element platziert werden soll. Da diese Liste nicht geordnet ist, ist es nicht wichtig, an welcher Stelle das neue Element in Bezug auf die anderen bereits in der Liste enthaltenen Elemente platziert wird. Der neue Gegenstand kann überall hinkommen. Vor diesem Hintergrund ist es sinnvoll, den neuen Gegenstand an einem möglichst einfachen Ort zu platzieren.

Es sei daran erinnert, dass uns die verknüpfte Listenstruktur nur einen einzigen Einstiegspunkt bietet, nämlich den Kopf der Liste. Alle anderen Knoten können nur erreicht werden, indem man auf den ersten Knoten zugreift und dann den <code>next</code> Link folgt. Das bedeutet, dass der einfachste Ort zum Hinzufügen des neuen Knotens direkt am Kopf oder am Anfang der Liste liegt. Mit anderen Worten, wir machen das neue Element zum ersten Element der Liste, und die vorhandenen Elemente müssen mit diesem neuen ersten Element verknüpft werden, damit sie folgen.

Die in Abbildung 6 gezeigte verknüpfte Liste wurde durch mehrmaligen Aufruf der <code>add</code>-Methode erstellt.

In [8]:
mylist.add(31)
mylist.add(77)
mylist.add(17)
mylist.add(93)
mylist.add(26)
mylist.add(54)

Beachten Sie, dass 31 das erste Element ist, das der Liste hinzugefügt wird, und dass es schließlich der letzte Knoten auf der verknüpften Liste sein wird, da jedes andere Element vor ihm hinzugefügt wird. Da 54 das zuletzt hinzugefügte Element ist, wird es auch zum Datenwert im ersten Knoten der verknüpften Liste.

Die <code>add</code>-Methode ist in Listing 4 dargestellt. Jedes Element der Liste muss sich in einem Knotenobjekt befinden. Zeile 2 erstellt einen neuen Knoten und platziert das Element als seine Daten. Jetzt müssen wir den Prozess abschließen, indem wir den neuen Knoten mit der bestehenden Struktur verknüpfen. Dies erfordert zwei Schritte, wie in Abbildung 7 dargestellt. In Schritt 1 (Zeile 3) wird die <code>next</code> Referenz des neuen Knotens so geändert, dass sie sich auf den alten ersten Knoten der Liste bezieht. Nun, da der Rest der Liste ordnungsgemäß an den neuen Knoten angehängt wurde, können wir den Kopf der Liste so ändern, dass er sich auf den neuen Knoten bezieht. Die Zuweisungsanweisung in Zeile 4 setzt den Kopf der Liste.

Die Reihenfolge der beiden oben beschriebenen Schritte ist sehr wichtig. Was geschieht, wenn die Reihenfolge von Zeile 3 und Zeile 4 umgekehrt wird? Wenn die Änderung des Listenkopfes zuerst geschieht, ist das Ergebnis in Abbildung 8 zu sehen. Da der Kopf die einzige externe Referenz zu den Listenknoten war, gehen alle ursprünglichen Knoten verloren und können nicht mehr aufgerufen werden.

**Listing 4**

In [9]:
def add(self,item):
    temp = Node(item)
    temp.setNext(self.head)
    self.head = temp

<img src="Bilder/ADT/adt_lists_fig7.PNG" width=400px>

<center><b>Abbildung 7:</b> Das Hinzufügen eines neuen Knotens ist ein zweistufiger Prozess</center>
<img src="Bilder/ADT/adt_lists_fig8.PNG" width=400px>

<center><b>Abbildung 8:</b> Ergebnis der Umkehrung der Reihenfolge der beiden Schritte</center>

Die nächsten Methoden, die wir implementieren werden - <code>size</code>, <code>search</code> <code>remove</code> - basieren alle auf einer Technik, die als "Linked List Traversal" bekannt ist. Traversal bezieht sich auf den Prozess des systematischen Besuchs jedes Knotens. Dazu verwenden wir eine externe Referenz, die am ersten Knoten in der Liste beginnt. Wenn wir jeden Knoten besuchen, verschieben wir den Verweis auf den nächsten Knoten, indem wir den nächsten Verweis "überqueren".

Um die <code>size</code>-Methode zu implementieren, müssen wir die verknüpfte Liste durchlaufen und die Anzahl der aufgetretenen Knoten zählen. Listing 5 zeigt den Python-Code für die Zählung der Anzahl der Knoten in der Liste. Die externe Referenz wird als <code>currrent</code> bezeichnet und wird mit dem Kopf der Liste in Zeile 2 initialisiert. Zu Beginn des Prozesses haben wir keine Knoten gesehen, daher wird die Zählung auf 0 gesetzt. Die Zeilen 4-6 implementieren tatsächlich die Überquerung. Solange die aktuelle Referenz noch nicht das Ende der Liste (<code>None</code>) gesehen hat, bewegen wir current entlang zum nächsten Knoten über die Zuweisungsanweisung in Zeile 6. Auch hier ist die Möglichkeit, eine Referenz mit <code>None</code> zu vergleichen, sehr nützlich. Jedes Mal, wenn <code>current</code> sich zu einem neuen Knoten bewegt, fügen wir 1 zu <code>count</code> hinzu. Schließlich wird <code>count</code> zurückgegeben, nachdem die Iteration gestoppt wurde. Abbildung 9 zeigt diesen Prozess, während er in der Liste nach unten verläuft.

**Listing 5**

In [10]:
def size(self):
    current = self.head
    count = 0
    while current != None:
        count = count + 1
        current = current.getNext()

    return count

<img src="Bilder/ADT/adt_lists_fig9.PNG" width=400px>

<center><b>Abbildung 9:</b> Durchqueren der Verknüpfungsliste vom Kopf bis zum Ende</center>

Bei der Suche nach einem Wert in einer verknüpften Liste wird bei der Implementierung einer ungeordneten Liste ebenfalls die Traversaltechnik verwendet. Wenn wir jeden Knoten in der verknüpften Liste besuchen, werden wir fragen, ob die dort gespeicherten Daten mit dem gesuchten Element übereinstimmen. In diesem Fall müssen wir jedoch möglicherweise nicht den ganzen Weg bis zum Ende der Liste durchqueren. Wenn wir tatsächlich zum Ende der Liste gelangen, bedeutet dies, dass das gesuchte Element nicht vorhanden sein darf. Wenn wir den gesuchten Gegenstand finden, brauchen wir auch nicht fortzufahren.

Listing 6 zeigt die Umsetzung der <code>search</code>-Methode. Wie bei der <code>size</code>-Methode wird die Durchquerung so initialisiert, dass sie am Anfang der Liste (Zeile 2) beginnt. Wir verwenden auch eine boolesche Variable namens <code>found</code>, um uns zu merken, ob wir den gesuchten Gegenstand gefunden haben. Da wir das Element zu Beginn der Durchquerung nicht gefunden haben, kann <code>found</code> auf <code>False</code> gesetzt werden (Zeile 3). Die Iteration in Zeile 4 berücksichtigt beide oben diskutierten Bedingungen. Solange es noch weitere Knoten zu besuchen gibt und wir den gesuchten Gegenstand nicht gefunden haben, prüfen wir den nächsten Knoten weiter. Die Frage in Zeile 5 fragt, ob das Datenelement im aktuellen Knoten vorhanden ist. Wenn ja, kann <code>found</code> auf <code>True</code> gesetzt werden.

**Listing 6**

In [11]:
def search(self,item):
    current = self.head
    found = False
    while current != None and not found:
        if current.getData() == item:
            found = True
        else:
            current = current.getNext()

    return found

Als Beispiel suchen wir in der Liste ``mylist``nach dem Element 17 (vorhanden) und dem Element 42 (nicht vorhanden).

In [12]:
elem1 = 17
elem2 = 42
print('Element',elem1,'in Liste gefunden:',mylist.search(elem1))
print('Element',elem2,'in Liste gefunden:',mylist.search(elem2))

Element 17 in Liste gefunden: True
Element 42 in Liste gefunden: False


Da 17 in der Liste steht, muss der Traversierungsprozess nur zu dem Knoten mit 17 gehen. An diesem Punkt wird die gefundene Variable auf <code>True</code> gesetzt und die <code>while</code>-Bedingung wird fehlschlagen, was zu dem oben gesehenen Rückgabewert führt. Dieser Prozess ist in Abbildung 10 zu sehen.

<img src="Bilder/ADT/adt_lists_fig10.PNG" width=400px>

<center><b>Abbildung 10:</b> Erfolgreiche Suche nach dem Wert 17</center>

Die <code>remove</code>-Methode erfordert zwei logische Schritte. Zunächst müssen wir die Liste nach dem zu entfernenden Element durchgehen. Sobald wir den Gegenstand gefunden haben (erinnern Sie sich, dass wir annehmen, dass er vorhanden ist), müssen wir ihn entfernen. Der erste Schritt ist der Suche sehr ähnlich. Ausgehend von einem externen Verweis auf den Kopf der Liste durchlaufen wir die Links, bis wir das gesuchte Objekt finden. Da wir davon ausgehen, dass dieses Element vorhanden ist, wissen wir, dass die Iteration beendet wird, bevor der aktuelle Wert auf <code>None</code> steht. Das bedeutet, dass wir einfach den in der Bedingung <code>found</code> verwenden können.

Wenn <code>found</code> <code>True</code> wird, ist <code>current</code> ein Verweis auf den Knoten, der das zu entfernende Element enthält. Aber wie können wir es entfernen? Eine Möglichkeit wäre, den Wert des Elements durch eine Markierung zu ersetzen, die anzeigt, dass das Element nicht mehr vorhanden ist. Das Problem bei diesem Ansatz ist, dass die Anzahl der Knoten nicht mehr mit der Anzahl der Elemente übereinstimmt. Es wäre viel besser, das Element durch Entfernen des gesamten Knotens zu entfernen.

Um den Knoten, der das Element enthält, zu entfernen, müssen wir die Verknüpfung im vorherigen Knoten so ändern, dass sie auf den Knoten verweist, der nach dem aktuellen Knoten kommt. Leider gibt es keine Möglichkeit, in der verknüpften Liste rückwärts zu gehen. Da sich current auf den Knoten vor dem Knoten bezieht, an dem wir die Änderung vornehmen möchten, ist es zu spät, um die erforderliche Änderung vorzunehmen.

Die Lösung für dieses Dilemma besteht darin, zwei externe Verweise zu verwenden, während wir die verknüpfte Liste durchlaufen. <code>current</code> verhält sich genauso wie zuvor und markiert die aktuelle Position des Durchlaufs. Der neue Verweis, den wir vorhergehend nennen werden, wird immer einen Knoten hinter current durchlaufen. Auf diese Weise, wenn <code>current</code> an dem zu entfernenden Knoten anhält, verweist previous auf die richtige Stelle in der verknüpften Liste für die Änderung.

Listing 7 zeigt die vollständige <code>remove</code>-Methode. Die Zeilen 2-3 weisen den beiden Verweisen Anfangswerte zu. Beachten Sie, dass <code>current</code> wie in den anderen Verfahrbeispielen am Listenkopf beginnt. <code>previous</code> hingegen wird angenommen, dass es immer einen Knoten hinter <code>current</code> fährt. Aus diesem Grund beginnt <code>previous</code> mit einem Wert von <code>None</code>, da vor dem Kopf kein Knoten vorhanden ist (siehe Abbildung 11). Die boolesche Variable <code>found</code> wird wiederum zur Steuerung der Iteration verwendet.

In den Zeilen 6-7 fragen wir, ob das im aktuellen Knoten gespeicherte Element das Element ist, das wir entfernen möchten. Wenn ja, kann <code>found</code> auf True gesetzt werden. Wenn wir das Element nicht finden, müssen sowohl <code>previous</code> als auch <code>current</code> um einen Knoten nach vorne verschoben werden. Auch hier ist die Reihenfolge dieser beiden Aussagen entscheidend. <code>previous</code> muss zuerst um einen Knoten nach vorne an die Stelle <code>current</code> verschoben werden. Dann kann der <code>current</code> verschoben werden. Dieser Vorgang wird oft als "Inch-Worming" bezeichnet, da <code>previous</code> <code>current</code> einholen muss, bevor <code>current</code>vorwärts bewegt wird. Abbildung 12 zeigt die Bewegung von <code>previous</code> und <code>current</code>, während sie die Liste nach unten durchlaufen und nach dem Knoten mit dem Wert 17 suchen.

**Listing 7**

In [13]:
def remove(self,item):
    current = self.head
    previous = None
    found = False
    while not found:
        if current.getData() == item:
            found = True
        else:
            previous = current
            current = current.getNext()

    if previous == None:
        self.head = current.getNext()
    else:
        previous.setNext(current.getNext())

<img src="Bilder/ADT/adt_lists_fig11.PNG" width=400px>

<center><b>Abbildung 11:</b> Anfangswerte für die Referenzen <code>previous</code> und <code>current</code></center>

<img src="Bilder/ADT/adt_lists_fig12.PNG" width=400px>

<center><b>Abbildung 12:</b> <code>previous</code> und <code>current</code> bewegen sich die Liste nach unten</center>

Sobald der Suchschritt des Entfernens abgeschlossen ist, müssen wir den Knoten aus der verknüpften Liste entfernen. Abbildung 13 zeigt die Verknüpfung, die geändert werden muss. Es gibt jedoch einen Sonderfall, der behandelt werden muss. Wenn das zu entfernende Element zufällig das erste Element in der Liste ist, dann wird Current auf den ersten Knoten in der verknüpften Liste verweisen. Dies bedeutet auch, dass <code>previous</code> <code>None</code> sein wird. Wir sagten bereits, dass <code>previous</code> sich auf den Knoten bezieht, dessen nächster Verweis geändert werden muss, um das Entfernen abzuschließen. In diesem Fall ist es nicht <code>previous</code>, sondern vielmehr der Kopf der Liste, der geändert werden muss (siehe Abbildung 14).

<img src="Bilder/ADT/adt_lists_fig13.PNG" width=400px>

<center><b>Abbildung 13:</b> Ein Element aus der Mitte der Liste entfernen</center>

<img src="Bilder/ADT/adt_lists_fig14.PNG" width=400px>

<center><b>Abbildung 14:</b> Entfernen des ersten Knotens aus der Liste</center>

Zeile 12 ermöglicht es uns zu prüfen, ob wir es mit dem oben beschriebenen Sonderfall zu tun haben. Wenn vorheriger sich nicht bewegt hat, hat er immer noch den Wert <code>None</code>, wenn <code>found</code> <code>True</code> wird. In diesem Fall (Zeile 13) wird der Listenkopf so modifiziert, dass er sich auf den Knoten nach dem aktuellen Knoten bezieht, wodurch der erste Knoten aus der verknüpften Liste entfernt wird. Wenn vorheriger jedoch nicht <code>None</code> ist, befindet sich der zu entfernende Knoten irgendwo unterhalb der verknüpften Listenstruktur. In diesem Fall liefert uns der vorherige Verweis den Knoten, dessen nächster Verweis geändert werden muss. Zeile 15 verwendet die <code>setNext</code>-Methode von <code>previous</code>, um die Entfernung durchzuführen. Beachten Sie, dass in beiden Fällen das Ziel der Verweisänderung <code>current.getNext()</code> ist. Eine Frage, die sich oft stellt, ist, ob die beiden hier gezeigten Fälle auch die Situation behandeln, in der sich das zu entfernende Element im letzten Knoten der verknüpften Liste befindet. Wir überlassen es Ihnen, dies zu berücksichtigen.

---
### ADT04-01 Praktische Übung (optional)
Die übrigen Methoden ``append``, ``insert``, ``index`` und ``pop`` werden als praktische Übungen überlassen. Denken Sie daran, dass bei jeder dieser Methoden berücksichtigt werden muss, ob die Änderung am Anfang der Liste oder an anderer Stelle stattfindet. Auch bei ``insert``,``index`` und ``pop`` müssen wir die Positionen der Liste  angeben. Gehen Sie davon aus, dass die Positionsnamen ganze Zahlen sind, die mit 0 beginnen.

---
## Der abstrakte Datentyp *geordnete Liste*

Wir werden nun eine Art von Liste betrachten, die als geordnete Liste bezeichnet wird. Wäre zum Beispiel die oben gezeigte Liste von ganzen Zahlen eine geordnete Liste (in aufsteigender Reihenfolge), dann könnte sie als 17, 26, 31, 54, 77 und 93 geschrieben werden. Da 17 das kleinste Element ist, nimmt es die erste Position in der Liste ein. Da 93 der größte Eintrag ist, nimmt er die letzte Position ein.

Die Struktur einer geordneten Liste ist eine Sammlung von Elementen, bei der jedes Element eine relative Position einnimmt, die auf einem zugrundeliegenden Merkmal des Elements beruht. Die Reihenfolge ist in der Regel entweder aufsteigend oder absteigend, und wir gehen davon aus, dass die Listenelemente eine sinnvolle Vergleichsoperation haben, die bereits definiert ist. Viele der geordneten Listenoperationen sind die gleichen wie die der ungeordneten Liste.

* <code>OrderedList()</code> erstellt eine neue geordnete Liste, die leer ist. Sie benötigt keine Parameter und gibt eine leere Liste zurück.
* <code>add(item)</code> fügt der Liste ein neues Element hinzu und stellt sicher, dass die Reihenfolge erhalten bleibt. Es benötigt das Element und gibt nichts zurück. Angenommen, das Element ist nicht bereits in der Liste enthalten.
* <code>remove(item)</code> entfernt das Element aus der Liste. Es benötigt das Element und modifiziert die Liste. Angenommen, das Element ist in der Liste vorhanden.
* <code>search(item)</code> sucht nach dem Element in der Liste. Es benötigt das Element und gibt einen booleschen Wert zurück.
* <code>isEmpty()</code> testet, ob die Liste leer ist. Sie benötigt keine Parameter und gibt einen booleschen Wert zurück.
* <code>size()</code> gibt die Anzahl der Elemente in der Liste zurück. Sie benötigt keine Parameter und gibt eine ganze Zahl zurück.
* <code>index(item)</code> gibt die Position des Elements in der Liste zurück. Es benötigt das Element und gibt den Index zurück. Angenommen, das Element befindet sich in der Liste.
* <code>pop()</code> entfernt und gibt das letzte Element in der Liste zurück. Es benötigt nichts und gibt ein Element zurück. Angenommen, die Liste enthält mindestens ein Element
* <code>pop(pos)</code> entfernt und gibt das Element an der Position pos zurück. Er benötigt die Position und gibt den Artikel zurück. Angenommen, das Element befindet sich in der Liste.

---
### Implementierung der geordneten Liste

Um die geordnete Liste zu implementieren, müssen wir daran denken, dass die relativen Positionen der Positionen auf einem zugrundeliegenden Merkmal beruhen. Die geordnete Liste der oben angegebenen ganzen Zahlen (17, 26, 31, 54, 77 und 93) kann durch eine verknüpfte Struktur dargestellt werden, wie in Abbildung 15 gezeigt. Auch hier eignet sich die Knoten- und Verknüpfungsstruktur ideal zur Darstellung der relativen Positionierung der Elemente.

<img src="Bilder/ADT/adt_lists_fig15.PNG" width=400px>

<center><b>Abbildung 15:</b> Eine geordnete Liste</center>

Um die Klasse <code>OrderedList</code> zu implementieren, werden wir die gleiche Technik verwenden, wie wir sie zuvor bei ungeordneten Listen gesehen haben. Auch hier wird eine leere Liste durch einen Kopfverweis auf <code>None</code> gekennzeichnet (siehe Listing 8).

**Listing 8**

In [14]:
class OrderedList:
    def __init__(self):
        self.head = None

Bei der Betrachtung der Operationen für die geordnete Liste ist zu beachten, dass die <code>isEmpty</code>- und <code>size</code>-Methoden wie bei ungeordneten Listen implementiert werden können, da sie nur die Anzahl der Knoten in der Liste ohne Rücksicht auf die tatsächlichen Werte der Elemente behandeln. Ebenso wird die <code>remove</code>-Methode einwandfrei funktionieren, da wir das Element noch finden und dann um den Knoten herum verlinken müssen, um es zu entfernen. Die beiden verbleibenden Methoden, Suchen und Hinzufügen, erfordern einige Änderungen.

Bei der Suche in einer ungeordneten verknüpften Liste mussten wir die Knoten einzeln durchlaufen, bis wir entweder das gesuchte Element gefunden haben oder uns die Knoten ausgegangen sind (<code>None</code>). Es stellt sich heraus, dass der gleiche Ansatz auch mit der geordneten Liste funktionieren würde, und in dem Fall, wo wir den Gegenstand finden, ist er genau das, was wir brauchen. Sollte das Element jedoch nicht in der Liste enthalten sein, können wir den Vorteil der Ordnung nutzen, um die Suche so schnell wie möglich zu beenden.

Abbildung 16 zeigt zum Beispiel die geordnete verlinkte Liste, während <code>dearch</code> nach dem Wert 45 sucht. Beim Durchlaufen, beginnend am Anfang der Liste, vergleichen wir zunächst mit 17. Da 17 nicht der gesuchte Wert ist, gehen wir zum nächsten Knoten über, in diesem Fall 26. Auch dies ist nicht das gesuchte Element, also gehen wir weiter zu 31 und dann zu 54. Da 54 nicht das Element ist, das wir suchen, wäre unsere alte Strategie, weiterzugehen. Da es sich jedoch um eine geordnete Liste handelt, ist dies nicht notwendig. Sobald der Wert im Knoten größer wird als der Gesuchte, kann die Suche abbrechen und <code>False</code> zurückgeben. Es besteht keine Möglichkeit, dass das Element weiter hinten in der verknüpften Liste existiert.

<img src="Bilder/ADT/adt_lists_fig16.PNG" width=400px>

<center><b>Abbildung 16:</b> Suche in einer geordneten Liste</center>

Listing 9 zeigt die vollständige Suchmethode. Es ist einfach, die oben besprochene neue Bedingung einzubeziehen, indem man eine weitere boolesche Variable, <code>stop</code>, hinzufügt und sie auf <code>False</code> (Zeile 4) initialisiert. Solange <code>stop</code> <code>False</code> ist, können wir in der Liste (Zeile 5) weiter nach vorne schauen. Wenn jemals ein Knoten entdeckt wird, der Daten enthält, die größer sind als das gesuchte Element, setzen <code>stop</code> auf <code>True</code> (Zeile 9-10). Die übrigen Zeilen sind identisch mit der ungeordneten Listensuche.

**Listing 9**

In [15]:
def search(self,item):
    current = self.head
    found = False
    stop = False
    while current != None and not found and not stop:
        if current.getData() == item:
            found = True
        else:
            if current.getData() > item:
                stop = True
            else:
                current = current.getNext()

    return found

Die bedeutendste Änderung geschieht in <code>add</code>. Erinnern Sie sich daran, dass bei nicht geordneten Listen die <code>add</code>-Methode einfach einen neuen Knoten an den Anfang der Liste setzen könnte. Das war der einfachste Zugriffspunkt. Leider wird dies bei geordneten Listen nicht mehr funktionieren. Es ist nun notwendig, dass wir die spezifische Stelle entdecken, an die ein neues Element in die bestehende geordnete Liste gehört.

Angenommen, wir haben die geordnete Liste, die aus 17, 26, 54, 77 und 93 besteht, und wir wollen den Wert 31 hinzufügen. Die Additionsmethode muss entscheiden, dass der neue Artikel zwischen 26 und 54 liegt. Abbildung 17 zeigt den Aufbau, den wir benötigen. Wie wir bereits erläutert haben, müssen wir die verknüpfte Liste durchgehen und nach dem Ort suchen, an dem der neue Knoten hinzugefügt werden soll. Wir wissen, dass wir diesen Ort gefunden haben, wenn uns entweder die Knoten ausgehen (der aktuelle Knoten wird zu <code>None</code>) oder der Wert des aktuellen Knotens größer wird als das Element, das wir hinzufügen möchten. In unserem Beispiel veranlasst uns der Wert 54 zum Aufhören.

<img src="Bilder/ADT/adt_lists_fig17.PNG" width=400px>

<center><b>Abbildung 17:</b> Hinzufügen eines Eintrags zu einer geordneten verknüpften Liste</center>

Wie wir bei den ungeordneten Listen gesehen haben, ist es notwendig, einen zusätzlichen Verweis zu haben, der wiederum <code>previous</code> genannt wird, da <code>current</code> keinen Zugang zu dem Knoten bietet, der geändert werden muss. Listing 10 zeigt die vollständige <code>add</code>-Methode. Die Zeilen 2-3 bilden die beiden externen Referenzen und die Zeilen 9-10 ermöglichen es wiederum, dass <code>previous</code> jedes Mal während der Iteration einen Knoten hinter <code>current</code> folgt. Die Bedingung (Zeile 5) erlaubt es, die Iteration so lange fortzusetzen, solange es noch Knoten gibt und der Wert im aktuellen Knoten nicht größer als das Element ist. In jedem Fall haben wir, wenn die Iteration fehlschlägt, die Stelle für den neuen Knoten gefunden.

Der Rest der Methode schließt den in Abbildung 17 dargestellten zweistufigen Prozess ab. Sobald ein neuer Knoten für das Element erstellt worden ist, bleibt nur noch die Frage, ob der neue Knoten am Anfang der verknüpften Liste oder an einer Stelle in der Mitte hinzugefügt wird. Auch hier kann das <code>previous == None</code> (Zeile 13) zur Beantwortung der Frage verwendet werden.

**Listing 10**

In [16]:
def add(self,item):
    current = self.head
    previous = None
    stop = False
    while current != None and not stop:
        if current.getData() > item:
            stop = True
        else:
            previous = current
            current = current.getNext()

    temp = Node(item)
    if previous == None:
        temp.setNext(self.head)
        self.head = temp
    else:
        temp.setNext(current)
        previous.setNext(temp)

---
## Analyse von verknüpften Listen

Um die Komplexität der verknüpften Listenoperationen zu analysieren, müssen wir prüfen, ob sie durchlaufen werden müssen. Betrachten Sie eine verknüpfte Liste, die $n$ Knoten hat. Die <code>isEmpty</code>-Methode ist O(1), da ein Schritt erforderlich ist, um die Kopfreferenz auf <code>None</code> zu überprüfen. <code>size</code> hingegen wird immer n Schritte erfordern, da es keine Möglichkeit gibt, zu wissen, wie viele Knoten sich in der verknüpften Liste befinden, ohne vom Kopf zum Ende zu traversieren. Daher ist die Länge $O(n)$. Das Hinzufügen eines Eintrags zu einer ungeordneten Liste wird immer $O(1)$ sein, da wir den neuen Knoten einfach an den Kopf der verknüpften Liste setzen. Das Suchen und Entfernen sowie das Hinzufügen für eine geordnete Liste erfordern jedoch alle den Traversierungsprozess. Obwohl sie im Durchschnitt nur die Hälfte der Knoten durchlaufen müssen, sind diese Methoden alle $O(n)$, da im schlimmsten Fall jeder jeden Knoten in der Liste bearbeitet.

Sie haben vielleicht auch bemerkt, dass diese Implementierung von der tatsächlichen Definiton von Python-Listen abweicht, die zuvor angegeben wurde. D.h. verknüpfte Listen funktionieren in Python anders, denn die tatsächliche Implementierung einer Python-Liste basiert auf dem Konzept eines Arrays. Wir besprechen dies in Kapitel 8 ausführlicher.

---
### Zusammenfassung

* Lineare Datenstrukturen halten ihre Daten in geordneter Form.
* Stapel sind einfache Datenstrukturen, die eine LIFO-, last-in-first-out-Ordnung pflegen.
* Die grundlegenden Operationen für einen Stack sind <code>push</code>, <code>pop</code> und <code>isEmpty</code>.
* Warteschlangen sind einfache Datenstrukturen, die eine FIFO-Ordnung (First-In-First-Out) aufrechterhalten.
* Die grundlegenden Operationen für eine Warteschlange sind <code>enqueue</code>, <code>enqueue</code> und <code>isEmpty</code>.
* Präfix, Infix und Postfix sind alles Möglichkeiten, Ausdrücke zu schreiben.
* Stapel sind sehr nützlich für den Entwurf von Algorithmen zur Auswertung und Übersetzung von Ausdrücken.
* Stacks können ein Umkehrmerkmal bieten.
* Warteschlangen können bei der Konstruktion von Timing-Simulationen helfen.
* Simulationen verwenden Zufallszahlengeneratoren, um eine Situation aus dem wirklichen Leben zu erzeugen und die Beantwortung von "Was-wäre-wenn"-Fragen zu ermöglichen.
* Deques sind Datenstrukturen, die ein hybrides Verhalten wie das von Stacks und Warteschlangen ermöglichen.
* Die grundlegenden Operationen für eine Deque sind <code>addFront</code>, <code>addRear</code>, <code>removeFront</code>, <code>removeRear</code> und <code>isEmpty</code>.
* Listen sind Sammlungen von Elementen, bei denen jedes Element eine relative Position einnimmt.
* Eine verknüpfte Listenimplementierung erhält die logische Reihenfolge aufrecht, ohne dass physische Speicheranforderungen erforderlich sind.
* Die Modifizierung des Kopfes der verknüpften Liste ist ein Sonderfall.